# QUDO a n vecinos

Autor: Alejandro Mata Ali (ITCL)

En este notebook vamos a resolver el problema QUDO de $n$ vecinos definido como aquel problema de minimización con $x_i$ enteros entre $0$ y $d_i-1$, tal que la función de coste es:

$$C(\vec{x})=\sum_{i=0}^{N-1}\sum_{j=i}^{\min(N-1,i+n)} W_{ij} x_{i}x_{j}$$

La interacción será solo entre las variables $i$ y las $n$ anteriores a ella. Esto es, la matriz $W_{ij}$ será triangular superior con $n$ diagonales además de la principal.

In [2]:
import numpy as np
import itertools
from time import time

# Función creadora de la red

Función que genera la red.

In [3]:
def generate_tensor_network(W_matrix:np.array, n_vecinos:int, dimensiones:np.array, tau:float)->list:
    _n_variables = len(W_matrix)
    tensor_list = [None,]*_n_variables
    # Creamos el tensor inicial, de medicion
    tensor_list[0] = tensor_initial_generator(0, n_vecinos, None, W_matrix, dimensiones, tau)

    # Creamos los demas tensores
    for _position in range(1,_n_variables-1):
        tensor_list[_position] = tensor_intermedio_generator(_position, n_vecinos, W_matrix, dimensiones, tau)
    
    # Creamos el tensor final
    tensor_list[-1] = tensor_final_generator(_n_variables, n_vecinos, W_matrix, dimensiones, tau)

    return tensor_list

## Funciones generadoras de tensores

In [4]:
def tensor_initial_generator(position, n_vecinos, solution, W_matrix, dimensiones, tau):
    # Primero determinamos el numero de señales y la dimension del grouping
    _n_señales = min(n_vecinos, position+1)
    _variable_lejana = position-_n_señales# Primera variable que recibe
    _dimensiones_futuras = dimensiones[_variable_lejana+1:position+1]# Dimensiones para los indices que transmitimos
    _shape = (dimensiones[position], productorio(_dimensiones_futuras))

    tensor = np.zeros(_shape, dtype=float)
    # Calculamos el bloque de coste derivado de las soluciones ya obtenidas
    if solution is None:
        _cost = 0
    else:
        _cost = W_matrix[_variable_lejana:position, position] @ solution[_variable_lejana:position]
    for i in range(dimensiones[position]):# Solo cambia el indice de la variable por determinar
        # Determinamos el valor del indice agrupado
        if solution is None:
            _index_list = [i]
        else:
            _index_list = solution[_variable_lejana+1:position] + [i]
        _index = group_index(_index_list, _dimensiones_futuras)
        # Hacemos la evolucion incluyendo el coste propio
        tensor[i,_index] = np.exp(-tau*(W_matrix[position,position]*i+_cost)*i)

    return tensor

def tensor_intermedio_generator(position, n_vecinos, W_matrix, dimensiones, tau):
    # Primero determinamos el numero de señales y la dimension del grouping
    _n_señales_up   = min(n_vecinos, position)
    _n_señales_down = min(n_vecinos, position+1)
    _variable_lejana_up   = position-_n_señales_up# Primera variable que recibe
    _variable_lejana_down = position-(_n_señales_down-1)# Primera variable que envia
    _dimensiones_pasadas = dimensiones[_variable_lejana_up:position]# Dimensiones para los indices que recibimos
    _dimensiones_futuras = dimensiones[_variable_lejana_down:position+1]# Dimensiones para los indices que transmitimos


    _shape = (productorio(_dimensiones_pasadas), productorio(_dimensiones_futuras))
    tensor = np.zeros(_shape, dtype=float)
    # Vamos añadiendo los elementos al tensor segun los indices posibles de entrada
    for _indexes in itertools.product(*map(range,_dimensiones_pasadas)):
        _cost = W_matrix[_variable_lejana_up:position, position] @ np.array(_indexes)
        for i in range(dimensiones[position]):# Indice propio de la variable
            # Determinamos el valor del indice agrupado
            # Entrada
            _up   = group_index(_indexes, _dimensiones_pasadas)
            # Salida
            _down = group_index(_indexes[1:]+(i,), _dimensiones_futuras)
            # Hacemos la evolucion incluyendo el coste propio
            tensor[_up, _down] = np.exp(-tau*(W_matrix[position,position]*i+_cost)*i)

    return tensor

def tensor_final_generator(n_variables, n_vecinos, W_matrix, dimensiones, tau):
    # Primero determinamos el numero de señales y la dimension del grouping
    position = n_variables-1
    _n_señales_up   = min(n_vecinos, position)
    _n_señales_down = min(n_vecinos, position+1)
    _variable_lejana_up   = position-_n_señales_up# Primera variable que recibe
    _variable_lejana_down = position-(_n_señales_down-1)# Primera variable que envia
    _dimensiones_pasadas = dimensiones[_variable_lejana_up:position]# Dimensiones para los indices que recibimos
    _dimensiones_futuras = dimensiones[_variable_lejana_down:position+1]# Dimensiones para los indices que transmitimos
    _shape = (productorio(_dimensiones_pasadas))
    tensor = np.zeros(_shape, dtype=float)
    # Vamos añadiendo los elementos al tensor segun los indices posibles de entrada
    for _indexes in itertools.product(*map(range,_dimensiones_pasadas)):
        _cost = W_matrix[_variable_lejana_up:position, position] @ np.array(_indexes)
        for i in range(dimensiones[position]):# Indice propio de la variable
            # Determinamos el valor del indice agrupado
            _up   = group_index(_indexes, _dimensiones_pasadas)
            # Hacemos la evolucion incluyendo el coste propio
            tensor[_up] += np.exp(-tau*(W_matrix[position,position]*i+_cost)*i)

    return tensor

# Función de contracción de la red

Función que genera la contracción de la primera iteración del algoritmo. Devuelve los tensores intermedios.

In [5]:
def contraction_tensors(lista_tensores:list):
    # Numero de tensores
    n_tensors = len(lista_tensores)
    # Creamos el primer tensor y lo normalizamos por estabilidad
    vector = lista_tensores[-1]
    vector /= np.linalg.norm(vector)
    # Creamos el primer elemento de la lista
    tensores_intermedios = [vector]
    append = tensores_intermedios.append

    # Contraemos cada tensor y almacenamos el resultado
    for i in range(n_tensors-2, -1, -1):
        vector = lista_tensores[i] @ vector
        # Normalizamos el vector usando la norma L2
        vector /= np.linalg.norm(vector)  # Normalizamos para que la suma sea 1
        append(vector)

    # Invertimos para facilitar el resto
    tensores_intermedios.reverse()
    return tensores_intermedios

# Funciones de auxiliares

In [6]:
def cost_QUDO(W_matrix:np.array, solution:np.array):
    '''Funcion que determina el coste de una solucion.'''
    _cost = 0
    for i, _variable_1 in enumerate(solution):
        for _j, _variable_2 in enumerate(solution[i:]):
            j = i+_j
            _cost += W_matrix[i,j]*_variable_1*_variable_2

    return _cost

def last_no_null(W_list:np.array)->int:
    '''Funcion que devuelve la posicion del ultimo elemento no nulo de un vector.'''
    for _i in range(len(W_list)-1, -1, -1):# La recorremos al reves
        if W_list[_i] != 0:
            return _i

def determinar_vecinos(W_matrix:np.array)->int:
    '''Funcion que determina el numero de vecino maximo del problema QUDO.'''
    _distancia_maxima = [ last_no_null(W_matrix[_i])-_i for _i in range(len(W_matrix)) ]

    return max(_distancia_maxima)
        
def determinar_ultima_variable(solution, W_matrix, dimensiones):
    '''Funcion que determina el valor optimo de la ultima variable en base a los valores de todas las demas.
    Ejecuta una comprobacion del coste de todas las posibilidades y escoge la de menor coste.'''
    _cost_list = np.zeros(dimensiones[-1])
    for _valor in range(dimensiones[-1]):
        _cost_list[_valor] = cost_QUDO(W_matrix, solution[:-1].copy()+[_valor])
    
    return np.argmin(_cost_list)

In [7]:
def productorio(vector:np.array):
    '''Funcion que hace el producto de los elementos de un vector.'''
    producto = 1
    for _componente in vector:
        producto *= _componente

    return producto

def group_index(index_list:list, dimensiones:list):
    '''Funcion que agrupa los indices teniendo en cuenta las dimensiones de cada uno'''
    index_value = 0
    _prefactor = 1
    for _i, _value in enumerate(index_list[::-1]):
        index_value += _prefactor*_value
        _prefactor *= dimensiones[-1-_i]# Multiplicamos el factor global de la agrupacion

    return index_value

# Función global

Función que recibe la matriz $W$ con $n$ vecinos, la dimensión de las diferentes variables y el $\tau$ de minimización, y devuelve el resultado como vector.

$x_i\in [0,dimensiones_i-1]$

In [8]:
def QUDO_Solver(W_matrix:np.array, dimensiones:np.array, tau:float=1)->list:
    # Determinamos a cuantos vecinos llega el modelo
    _n_vecinos   = determinar_vecinos(W_matrix)
    _n_variables = len(W_matrix)
    
    solution     = [0,]*_n_variables

    # Creamos la tensor network
    _tensor_network = generate_tensor_network(W_matrix, _n_vecinos, dimensiones, tau)

    # Contraemos la primera iteracion
    _tensores_intermedios = contraction_tensors(_tensor_network)

    # Obtenemos el valor correcto para esta variable
    solution[0] = np.argmax(_tensores_intermedios[0])
    # Por temas de accesos a memoria
    _tensores_intermedios.pop(0)
    _tensores_intermedios.pop(0)


    # Proceso iterativo para el resto de variables
    for _position in range(1, _n_variables-1):

        # Creamos el primer tensor de la cadena
        _tensor_inicial = tensor_initial_generator(_position, _n_vecinos, solution, W_matrix, dimensiones, tau)
        # Contraccion aprovechando los tensores intermedios
        _vector_de_salida = _tensor_inicial @ _tensores_intermedios[0]
        _tensores_intermedios.pop(0)
        print(_vector_de_salida)
        # Obtenemos el valor correcto para esta variable
        solution[_position] = np.argmax(abs(_vector_de_salida))

    # Obtenemos el valor de la última variable
    solution[-1] = determinar_ultima_variable(solution, W_matrix, dimensiones)

    return solution

# Pruebas

## Funciones auxiliares

In [9]:
def crea_instancia(n_variables:int, n_vecinos:int, values_range:tuple)->np.array:
    W_matrix = np.zeros((n_variables, n_variables), dtype=float)
    for i in range(n_variables):
        for j in range(i, min(i+n_vecinos+1,n_variables)):
            W_matrix[i,j] = ((values_range[1]-values_range[0])*np.random.rand()+values_range[0])
            W_matrix[j,i] = W_matrix[i,j]
    
    return W_matrix

In [10]:
def solver_aleatorio(W_matrix:np.array, dimensiones:np.array, n_iteraciones:int)->list[int]:
    _n_variables = len(W_matrix)
    solution = [0,]*_n_variables
    cost_solution = cost_QUDO(W_matrix, solution)
    for _i_iteracion in range(int(n_iteraciones)):
        solution_prueba = [ np.random.randint(0,_dim) for _dim in dimensiones ]
        cost_solution_prueba = cost_QUDO(W_matrix, solution_prueba)
        if cost_solution_prueba < cost_solution:
            cost_solution = cost_solution_prueba
            solution = solution_prueba.copy()

    return solution

In [11]:
def triangular_to_symmetric(triangular_matrix):
    n = triangular_matrix.shape[0]
    symmetric_matrix = np.zeros((n, n))
    
    # Copiar la matriz triangular inferior a la simétrica
    for i in range(n):
        for j in range(n):
            if i >= j:  # Parte inferior (incluyendo diagonal)
                symmetric_matrix[i, j] = triangular_matrix[i, j]
            else:  # Parte superior, reflejando la inferior
                symmetric_matrix[i, j] = triangular_matrix[j, i]
    
    return symmetric_matrix

## Hacemos pruebas

In [17]:
# Setup
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_auxiliar_functions import evaluar_qubo, generar_matriz_qubo, matrix_QUBO_to_dict
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_solvers import qubo_dimod_solver


n_variables  = 10
n_vecinos    = 4
dimensiones  = [3,]*n_variables
values_range = (0.,1.)

tau = 100
n_iteraciones = 1e3

# Creacion de la matriz
W_matrix = crea_instancia(n_variables, n_vecinos, values_range)
W_matrix /= np.linalg.norm(W_matrix)*n_vecinos*n_variables

# Resolvemos con la TN

np.random.seed(53)
# Generamos el caso

Q_matrix = generar_matriz_qubo(n_variables, n_vecinos + 1)
Q_matrix/= np.linalg.norm(Q_matrix)

Q_matrix_2 = triangular_to_symmetric(Q_matrix)
Q_matrix_dict = matrix_QUBO_to_dict(Q_matrix)
inicio = time()
solution_TN = QUDO_Solver(Q_matrix_2, dimensiones, tau=tau)
print("tiempo", time()-inicio)
print('Solucion TN:     ', ','.join(map(str,solution_TN)))
print('Coste solucion TN:     ', evaluar_qubo(Q_matrix, solution_TN))

solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))

[5.77350269e-01 1.67372707e-21 4.48346285e-54]
[4.15596964e-04 5.45829642e-29 2.12110162e-52]
[3.14270478e-83 7.02713420e-54 4.27628309e-48]
[1.00000000e+00 1.42813723e-36 8.17276352e-86]
[1.37759508e-57 1.78111060e-20 2.10947580e+11]
[5.85443992e-07 1.09948354e+32 2.52213435e+66]
[1.08262165e-003 5.59939258e-051 4.39483887e-102]
[9.47257769e-01 8.78940376e-07 1.40432818e-28]
tiempo 0.010807514190673828
Solucion TN:      0,0,0,2,0,2,2,0,0,1
Coste solucion TN:      -1.1697049512804316
Coste de dimod iter:  -0.2749838385196257
